## **Agents**

An agent-based workflow where LLMs act autonomously within a loop, interacting with their environment and receiving feedback to refine their actions and decisions.


<img src="../images/autonomous-agent.webp" alt="Autonomous Agents" width="500">

## **Use Cases:**
 
- Building a personal research assistant that autonomously searches academic papers, extracts key findings, and generates literature review summaries based on specific research questions.
- Creating an autonomous code reviewer that analyzes pull requests, identifies potential bugs and security issues, suggests improvements, and generates detailed review comments.
- Developing a customer support agent that handles inquiries by searching knowledge bases, generating appropriate responses, and escalating complex issues to human agents when needed.
- Managing social media presence by analyzing trending topics, generating relevant content, scheduling posts, and engaging with followers through personalized responses.
- Building an autonomous testing agent that generates test cases, executes tests, analyzes failures, and provides detailed bug reports with suggested fixes.
- Creating a data monitoring agent that continuously analyzes system metrics, detects anomalies, investigates root causes, and generates incident reports with recommended actions.

In [ ]:
%pip install openai pydantic --upgrade

In [2]:
import json
import os
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-proj-xxxx"

In [10]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL="gpt-4o"

In [15]:
# Here we define a sample function the agent can call.
# In practice, you might connect to real APIs or services for your agent's tasks.
def search_knowledge_base(query: str, options: dict) -> str:
    """
    Pretend to look for information about the provided query in a knowledge base.
    Returns a brief summary as a string.
    """
    # For demonstration, we'll return a placeholder string.
    # In a real scenario, you'd perform a search and generate a proper result.
    return f"Summary for '{query}': This is a simulated summary from the knowledge base."

# Next, we define our function schema to provide to the model.
tools = [{
    "type": "function",
    "function": {
        "name": "search_knowledge_base",
        "description": "Query a knowledge base to retrieve relevant info on a topic.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The user question or search query."
                },
                "options": {
                    "type": "object",
                    "properties": {
                        "num_results": {
                            "type": "number",
                            "description": "Number of top results to return."
                        },
                        "domain_filter": {
                            "type": [
                                "string",
                                "null"
                            ],
                            "description": "Optional domain to narrow the search (e.g. 'finance', 'medical'). Pass null if not needed."
                        },
                        "sort_by": {
                            "type": [
                                "string",
                                "null"
                            ],
                            "enum": [
                                "relevance",
                                "date",
                                "popularity",
                                "alphabetical"
                            ],
                            "description": "How to sort results. Pass null if not needed."
                        }
                    },
                    "required": [
                        "num_results",
                        "domain_filter",
                        "sort_by"
                    ],
                    "additionalProperties": False
                }
            },
            "required": [
                "query",
                "options"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

# We simulate a user asking the agent a question that might prompt the model to call the function.
messages = [
    {
        "role": "user",
        "content": "Hello, I'd like to know more about quantum computing. Could you give me a quick summary?"
    }
]

# Step 1: We call the model, providing the function tools. The model may decide to call the tool.
completion_1 = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools
)

In [18]:
# Step 2: The model might return a tool call. We'll check for any function calls and execute them.
tool_calls = completion_1.choices[0].message.tool_calls

results = []
if tool_calls:
    for call in tool_calls:
        function_name = call.function.name
        # Parse the JSON arguments:
        function_args = json.loads(call.function.arguments)
        
        if function_name == "search_knowledge_base":
            print('Calling function - ', function_name)
            search_result = search_knowledge_base(**function_args)
            # We'll store the result and associate it with this tool_call
            results.append(
                {
                    "tool_call_id": call.id,
                    "content": search_result
                }
            )

Calling function -  search_knowledge_base


In [19]:
# Step 3: We send the results back to the model so it can incorporate them into its final answer.
# We'll add both the original function call message and the tool's reply to our conversation.

if tool_calls and results:
    # Append the tool call message from the model
    messages.append(completion_1.choices[0].message)
    # Append our result(s) as 'tool' role messages
    for r in results:
        messages.append({
            "role": "tool",
            "tool_call_id": r["tool_call_id"],
            "content": r["content"]
        })

In [20]:
# Step 4: Make another call to the model, now that it has the tool's output.
completion_2 = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools
)

# Finally, we print out the model's final answer. This is how the user sees it.
final_answer = completion_2.choices[0].message.content
print("Final Answer from Agent:")
print(final_answer)

Final Answer from Agent:
Quantum computing is an advanced field of computing that uses the principles of quantum mechanics to process information. Unlike classical computers, which use bits as the smallest unit of data (0 or 1), quantum computers use quantum bits or qubits. Qubits can exist in multiple states simultaneously due to superposition, and they can be entangled, meaning the state of one qubit can depend on the state of another, even at a distance. This allows quantum computers to potentially solve complex problems much faster than classical computers, particularly in areas like cryptography, optimization, and simulation of quantum systems.


------------------------------

## An Alternative Approach

If an agent has finished all of it's tool calls, then it has likely finished with the task. 

Therefore we could also write the above code within a while loop that checks if the agent has finished all of it's tool calls. If it has, then we can break the loop.

In [22]:
def search_knowledge_base(query: str, options: dict) -> str:
    """
    Pretend to look for information about the provided query in a knowledge base.
    """
    return f"ChatGPT is a large language model developed by OpenAI."

In [25]:
messages = []
messages.append({"role": "user", "content": "Can you find information about ChatGPT in the AI knowledge base?"})

while True:
    completion = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )
    tool_calls = completion.choices[0].message.tool_calls
    if not tool_calls:
        break
    else: 
        for call in tool_calls:
            # Add the tool call to the messages
            messages.append(completion.choices[0].message)
            
            # Parse the JSON arguments:
            function_name = call.function.name
            function_args = json.loads(call.function.arguments)
            if function_name == "search_knowledge_base":
                print('Calling function - ', function_name)
                search_result = search_knowledge_base(**function_args)
                messages.append({
                    "role": "tool",
                    "tool_call_id": call.id,
                    "content": search_result
                })

Calling function -  search_knowledge_base
